## Step 1: Import libraries
First, we need to import the required libraries:

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.manifold import TSNE
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F

## Step 2: Define the Siamese Network architecture
We'll define a simple convolutional neural network (CNN) architecture and the Siamese network itself.

In [ ]:
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 32, 5),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(32, 64, 5),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
        )
        self.fc = nn.Sequential(
            nn.Linear(64 * 4 * 4, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
        )

    def forward_once(self, x):
        x = self.conv(x)
        x = x.view(-1, 64 * 4 * 4)
        x = self.fc(x)
        return x

    def forward(self, x1, x2):
        output1 = self.forward_once(x1)
        output2 = self.forward_once(x2)
        return output1, output2

 


## Step 3: Create the Contrastive Loss function
Now we'll create the contrastive loss function, which will measure the similarity between two output embedding

In [ ]:
class ContrastiveLoss(nn.Module):
    def __init__(self, margin):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, x1, x2, y):
        euclidean_distance = torch.nn.functional.pairwise_distance(x1, x2)
        loss = 0.5 * y * torch.pow(euclidean_distance, 2) + \
               0.5 * (1-y) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2)
        return loss.mean()

## Step 4: Load and preprocess the data
We'll use the MNIST dataset for our example. We need to create a custom dataset class to handle pairs of images and their corresponding labels.

In [ ]:
class SiameseMNIST(Dataset):
    def __init__(self, mnist_dataset):
        self.mnist_dataset = self.filter_dataset(mnist_dataset)

    def filter_dataset(self, dataset):
        filtered_data = []
        for img, label in dataset:
            if label == 4 or label == 9:
                filtered_data.append((img, label))
        return filtered_data

    def __getitem__(self, index):
        img1, label1 = self.mnist_dataset[index]
        label2 = label1
        while label2 == label1:
            index2 = torch.randint(len(self.mnist_dataset), (1,)).item()
            img2, label2 = self.mnist_dataset[index2]

        return img1, img2, torch.tensor(int(label1 == label2), dtype=torch.float32)

    def __len__(self):
        return len(self.mnist_dataset)


transform = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

siamese_train_dataset = SiameseMNIST(train_dataset)
siamese_test_dataset = SiameseMNIST(test_dataset)

train_loader = DataLoader(siamese_train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(siamese_test_dataset, batch_size=64, shuffle=True)


## Step 5: Train the Siamese Network
Next, we'll train the Siamese network using the contrastive loss function.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
siamese_net = SiameseNetwork().to(device)
criterion = ContrastiveLoss(margin=1.0)
optimizer = optim.Adam(siamese_net.parameters(), lr=0.001)

num_epochs = 10

for epoch in range(num_epochs):
    siamese_net.train()
    total_loss = 0.0
    for imgs1, imgs2, labels in train_loader:
        imgs1, imgs2, labels = imgs1.to(device), imgs2.to(device), labels.to(device)

        optimizer.zero_grad()
        output1, output2 = siamese_net(imgs1, imgs2)
        loss = criterion(output1, output2, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {total_loss / len(train_loader):.4f}')


Epoch [1/10], Loss: 0.0030
Epoch [2/10], Loss: 0.0000
Epoch [3/10], Loss: 0.0000
Epoch [4/10], Loss: 0.0000
Epoch [5/10], Loss: 0.0000
Epoch [6/10], Loss: 0.0000
Epoch [7/10], Loss: 0.0000
Epoch [8/10], Loss: 0.0000
Epoch [9/10], Loss: 0.0000
Epoch [10/10], Loss: 0.0000


## Step 6: Evaluate the Siamese Network
Finally, we'll evaluate the performance of the Siamese network by measuring the accuracy of the model on the test dataset.


In [ ]:
def compute_accuracy(loader, model, criterion, device):
    correct = 0
    total = 0
    with torch.no_grad():
        for imgs1, imgs2, labels in loader:
            imgs1, imgs2, labels = imgs1.to(device), imgs2.to(device), labels.to(device)

            output1, output2 = model(imgs1, imgs2)
            euclidean_distance = F.pairwise_distance(output1, output2)
            predictions = (euclidean_distance < criterion.margin).float()
            correct += (predictions == labels).sum().item()
            total += labels.size(0)

    return correct / total

siamese_net.eval()
accuracy = compute_accuracy(test_loader, siamese_net, criterion, device)
print(f'Accuracy on test dataset: {accuracy * 100:.2f}%')


Accuracy on test dataset: 100.00%
